In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
# This piece of code prompts to choose file
from google.colab import files
uploaded = files.upload()
# After choosing file use this piece of code
import io
application_df = pd.read_csv(io.BytesIO(uploaded['charity_data.csv']))
# application_df = pd.read_csv("../Resources/charity_data.csv")
application_df.head()

Saving charity_data.csv to charity_data.csv


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [3]:
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [4]:
# Drop the non-beneficial ID columns, 'EIN','STATUS' and 'SPECIAL_CONSIDERATIONS.
application_df = application_df.drop(['EIN','SPECIAL_CONSIDERATIONS','STATUS'],axis=1)
application_df

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,0,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1-9999,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,0,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1
...,...,...,...,...,...,...,...,...,...
34294,THE LIONS CLUB OF HONOLULU KAMEHAMEHA,T4,Independent,C1000,ProductDev,Association,0,5000,0
34295,INTERNATIONAL ASSOCIATION OF LIONS CLUBS,T4,CompanySponsored,C3000,ProductDev,Association,0,5000,0
34296,PTA HAWAII CONGRESS,T3,CompanySponsored,C2000,Preservation,Association,0,5000,0
34297,AMERICAN FEDERATION OF GOVERNMENT EMPLOYEES LO...,T5,Independent,C3000,ProductDev,Association,0,5000,1


In [5]:
# Look at AFFILIATION value counts for binning
aff_count = application_df['AFFILIATION'].value_counts() 
aff_count

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

In [6]:

affiliation_to_replace = list(aff_count[aff_count < 100].index)
# Replace in dataframe
for app in affiliation_to_replace :
  application_df['AFFILIATION'] = application_df['AFFILIATION'].replace(app,"Other")
# Check to make sure binning was successful
application_df['AFFILIATION'].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [7]:
name_count = application_df['NAME'].value_counts() 
name_count

PARENT BOOSTER USA INC                                                  1260
TOPS CLUB INC                                                            765
UNITED STATES BOWLING CONGRESS INC                                       700
WASHINGTON STATE UNIVERSITY                                              492
AMATEUR ATHLETIC UNION OF THE UNITED STATES INC                          408
                                                                        ... 
ST LOUIS SLAM WOMENS FOOTBALL                                              1
AIESEC ALUMNI IBEROAMERICA CORP                                            1
WEALLBLEEDRED ORG INC                                                      1
AMERICAN SOCIETY FOR STANDARDS IN MEDIUMSHIP & PSYCHICAL INVESTIGATI       1
WATERHOUSE CHARITABLE TR                                                   1
Name: NAME, Length: 19568, dtype: int64

In [8]:
name_to_replace = list(name_count[name_count < 5].index)
# Replace in dataframe
for app in name_to_replace :
  application_df['NAME'] = application_df['NAME'].replace(app,"Other")
# Check to make sure binning was successful
application_df['NAME'].value_counts()

Other                                                                19803
PARENT BOOSTER USA INC                                                1260
TOPS CLUB INC                                                          765
UNITED STATES BOWLING CONGRESS INC                                     700
WASHINGTON STATE UNIVERSITY                                            492
                                                                     ...  
GUILDS OF THE SANTA FE OPERA INC                                         5
FAMILY MOTOR COACH ASSOCIATION                                           5
JAMESTOWN SOCIETY                                                        5
MOST WORSHIPFUL PRINCE HALL GRAND LODGE OF FREE & ACCEPTED MASONS        5
PTA HAWAII CONGRESS                                                      5
Name: NAME, Length: 403, dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
application_count = application_df['APPLICATION_TYPE'].value_counts()
application_count

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(application_count[application_count < 1000].index)

# Replace in dataframe
for app in application_types_to_replace:
#   print(app)
  application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
classification_count = application_df['CLASSIFICATION'].value_counts()
classification_count

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# Choose a cutoff value and create a list of classifications to be replaced
classifications_to_replace = list(classification_count[classification_count < 1000].index)
# classifications_to_replace
# # Replace in dataframe
for cls in classifications_to_replace: 
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")  
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [13]:
# Convert categorical data to numeric with `pd.get_dummies`
X_dummies = pd.get_dummies(application_df)
len(X_dummies.columns)

438

In [14]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = X_dummies.drop('IS_SUCCESSFUL',axis=1).values
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
X_train.shape

(25724, 437)

In [15]:

scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [16]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_features = len(X_train[0])
hidden_layer1 =  100
hidden_layer2 = 30
hidden_layer3 = 20
nn = tf.keras.models.Sequential()
# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer1, activation="relu", input_dim=input_features))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer2, activation="sigmoid"))
# third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_layer3, activation="sigmoid"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               43800     
                                                                 
 dense_1 (Dense)             (None, 30)                3030      
                                                                 
 dense_2 (Dense)             (None, 20)                620       
                                                                 
 dense_3 (Dense)             (None, 1)                 21        
                                                                 
Total params: 47,471
Trainable params: 47,471
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [18]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 3s 3ms/step - loss: 0.4872 - accuracy: 0.7697
Epoch 2/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4355 - accuracy: 0.7965
Epoch 3/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4296 - accuracy: 0.7979
Epoch 4/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4258 - accuracy: 0.7988
Epoch 5/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4229 - accuracy: 0.8006
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4216 - accuracy: 0.8004
Epoch 7/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4199 - accuracy: 0.8012
Epoch 8/100
804/804 [==============================] - 4s 5ms/step - loss: 0.4187 - accuracy: 0.8005
Epoch 9/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4175 - accuracy: 0.8017
Epoch 10/100
804/804 [==============================] - 2s 3ms/step - loss: 0.4166 - accura

In [19]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.4611 - accuracy: 0.7897 - 712ms/epoch - 3ms/step
Loss: 0.46110907196998596, Accuracy: 0.7897375822067261


In [23]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimization.h5')
from google.colab import files
files.download("AlphabetSoupCharity_Optimization.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

LOGISTIC REGRESSION

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Fit the model
classifier = LogisticRegression().fit(X_train_scaled,y_train)

# Evaluate the model
y_pred = classifier.predict(X_test_scaled)
score = accuracy_score(y_test,y_pred)
score

0.7892711370262391

RANDOM FOREST CLASSIFIER

In [25]:
from sklearn.ensemble import RandomForestClassifier

# create and fit the model
clf = RandomForestClassifier(random_state=42, n_estimators=100).fit(X_train, y_train)

# Evaluate the model
y_predict = clf.predict(X_test_scaled)
score = accuracy_score(y_test,y_predict)
score

0.787405247813411